In [51]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [124]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 9

5


In [125]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=30, max_leaf_nodes=570, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-07)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(priors=None, var_smoothing=1e-09)'],
 ['[\'longest_win_streak_dif\', \'B_avg_TD_pct\', \'sub_dif\', \'B_total_rounds_fought\', "R_Women\'s Strawweight_rank", \'R_age\', \'B_Flyweight_rank\', "R_Women\'s Flyweight_rank", \'R_Lightweight_rank\', \'total_tit

In [126]:
df = pd.read_csv("../data/ufc-master.csv")

In [127]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [128]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [129]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [130]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4136
4136
4136


In [131]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Jordan Griffin,TJ Brown,-135,115,74.074074,115.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,7.0,0.00,1.0,3.0,0.0,0.0,4.0,0.0,0.0,0
201,Aalon Cruz,Spike Carlyle,-190,165,52.631579,165.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Blue,...,0.0,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
202,Sean Brady,Ismail Naurdiev,105,-125,105.000000,80.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,1.0,0.44,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0
203,Paul Felder,Dan Hooker,130,-150,130.000000,66.666667,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,5.0,0.00,0.2,0.0,0.0,0.0,1.0,0.0,0.0,1
204,Jim Crute,Michal Oleksiejczuk,100,-120,100.000000,83.333333,2020-02-22,"Auckland, New Zealand",New Zealand,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4331,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1
4332,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0
4333,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4334,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Stipe Miocic,Daniel Cormier,-106,-112,94.339623,89.285714,2020-08-15,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Sean O'Malley,Marlon Vera,-305,240,32.786885,240.000000,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Junior Dos Santos,Jairzinho Rozenstruik,115,-143,115.000000,69.930070,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,John Dodson,Merab Dvalishvili,188,-240,188.000000,41.666667,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Jim Miller,Vinc Pichel,100,-124,100.000000,80.645161,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Grant Dawson,Darrick Minner,-475,380,21.052632,380.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,1.0,0.66,1.0,2.0,2.0,6.0,0.0,0.0,1.0,0
196,Gabriel Silva,Kyler Phillips,-130,110,76.923077,110.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Blue,...,4.0,0.33,0.5,1.0,1.0,0.0,4.0,0.0,0.0,1
197,Brendan Allen,Tom Breese,110,-130,110.000000,76.923077,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,1.0,0.00,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
198,Luis Pena,Steve Garcia,-270,230,37.037037,230.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,1.0,0.00,1.0,1.0,1.0,3.0,0.0,0.0,0.0,0


In [132]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [133]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Stipe Miocic,Daniel Cormier,-106,-112,94.339623,89.285714,2020-08-15,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Sean O'Malley,Marlon Vera,-305,240,32.786885,240.000000,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Junior Dos Santos,Jairzinho Rozenstruik,115,-143,115.000000,69.930070,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,John Dodson,Merab Dvalishvili,188,-240,188.000000,41.666667,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Jim Miller,Vinc Pichel,100,-124,100.000000,80.645161,2020-08-15,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Grant Dawson,Darrick Minner,-475,380,21.052632,380.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,1.0,0.66,1.0,2.0,2.0,6.0,0.0,0.0,1.0,0
196,Gabriel Silva,Kyler Phillips,-130,110,76.923077,110.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Blue,...,4.0,0.33,0.5,1.0,1.0,0.0,4.0,0.0,0.0,1
197,Brendan Allen,Tom Breese,110,-130,110.000000,76.923077,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,1.0,0.00,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
198,Luis Pena,Steve Garcia,-270,230,37.037037,230.000000,2020-02-29,"Norfolk, Virginia, USA",USA,Red,...,1.0,0.00,1.0,1.0,1.0,3.0,0.0,0.0,0.0,0


In [134]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [135]:
models[1][model_num]

'GaussianNB(priors=None, var_smoothing=1e-09)'

In [136]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [137]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [138]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [139]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [140]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_7_25_20
GaussianNB()
['R_Reach_cms', 'total_round_dif', 'R_Height_cms', 'R_avg_SIG_STR_pct', 'B_age', 'R_longest_win_streak', 'lose_streak_dif', 'ko_dif', 'R_win_by_Decision_Majority', 'longest_win_streak_dif', 'avg_sub_att_dif', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
1


In [141]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [142]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [21]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 19862.479061934555
Feature: B_avg_SIG_STR_landed Score: 22312.81726390216
Feature: B_avg_TD_pct Score: 25586.00851843385
The best feature was B_avg_TD_pct.  It scored 25586.00851843385
Current best score is: 25586.00851843385
NO IMPROVEMENT
FINAL BEST SCORE: 25586.00851843385

model_4
GaussianNB()
['B_avg_TD_pct', 'height_dif', 'B_Reach_cms', 'B_wins', 'better_rank', 'sub_dif', 'B_Weight_lbs', 'R_total_rounds_fought', 'R_avg_TD_landed', 'B_Heavyweight_rank', 'B_Lightweight_rank', 'B_losses', 'total_round_dif', 'R_Height_cms', 'R_avg_SIG_STR_pct', 'B_age', 'R_longest_win_streak', 'lose_streak_dif', 'ko_dif', 'R_win_by_Decision_Majority', 'longest_win_streak_dif', 'avg_sub_att_dif', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
8



(3183, 34)

(3183,)

(3183, 2)

(200, 34)

...

(200, 33)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-106,-112,0.357799,0.642201,0
1,-305,240,0.930221,0.069779,1
2,115,-143,0.217338,0.782662,1
3,188,-240,0.092569,0.907431,1
4,100,-124,0.305829,0.694171,1
...,...,...,...,...,...
195,-475,380,0.997104,0.002896,0
196,-130,110,0.612679,0.387321,1
197,110,-130,0.999869,0.000131,0
198,-270,230,0.794277,0.205723,0



          Number of matches: 200
          Number of bets: 180
          Number of winning bets: 113
          Number of losing bets: 67
          Number of underdog bets: 60
          Number of underdog wins: 26
          Number of underdog losses: 34
          Number of Favorite bets: 113
          Number of favorite wins: 82
          Number of favorite losses: 31
          Number of even bets: 7
          Number of even wins: 5
          Number of even losses: 2
          Profit: 1636.5068742957055
          Profit per bet: 9.091704857198364
          Profit per match: 8.182534371478528
          
          
HI
HI
HI
HI
HI


In [143]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3193, 22)

(3193,)

(3193, 2)

(200, 22)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-106,-112,0.940706,0.059294,0
1,-305,240,0.809319,0.190681,1
2,115,-143,0.858357,0.141643,1
3,188,-240,0.131610,0.868390,1
4,100,-124,0.367304,0.632696,1
...,...,...,...,...,...
195,-475,380,0.971397,0.028603,0
196,-130,110,0.603612,0.396388,1
197,110,-130,0.999719,0.000281,0
198,-270,230,0.775955,0.224045,0



          Number of matches: 200
          Number of bets: 191
          Number of winning bets: 124
          Number of losing bets: 67
          Number of underdog bets: 71
          Number of underdog wins: 33
          Number of underdog losses: 38
          Number of Favorite bets: 112
          Number of favorite wins: 85
          Number of favorite losses: 27
          Number of even bets: 8
          Number of even wins: 6
          Number of even losses: 2
          Profit: 2811.306482263951
          Profit per bet: 14.718882106093982
          Profit per match: 14.056532411319754
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [144]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [145]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [146]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [147]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=30, max_leaf_nodes=570, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-07)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()'],
 ['[\'longest_win_streak_dif\', \'B_avg_TD_pct\', \'sub_dif\', \'B_total_rounds_fought\', "R_Women\'s Strawweight_rank", \'R_age\', \'B_Flyweight_rank\', "R_Women\'s Flyweight_rank", \'R_Lightweight_rank\', \'total_title_bout_dif\', \'better_rank\', 